In [1]:
from datetime import date
try:
    from . import scrapper
except:
    import scrapper


In [2]:
year = date.today().year
month = date.today().month
print(year, month)

2023 1


In [3]:
list_pick_members1 = ["1.1", "1.2", "1.3", "1.4", "1.5", "1.6", "1.7", "1.8", "1.9", "1.10", "1.11", "1.12", "1.14", "1.15"]
list_pick_members1_name = ["Canada", "Newfoundland and Labrador", "Prince Edward Island", "Nova Scotia", "New Brunswick", "Quebec", "Ontario", "Manitoba", "Saskatchewan", "Alberta", "British Columbia", "Yukon", "Northwest Territories", "Nunavut"]

list_pick_members2 = ["2.2", "2.3"]
list_pick_members2_name = ["Salaried employees paid a fixed salary", "Employees paid by the hour"]


joined_pi = zip(list_pick_members1, list_pick_members1_name)
joined_pi2 = zip(list_pick_members2, list_pick_members2_name)

pid=str(1410020101)
pickMembers1=str(1.3)
pickMembers2=str(2.1)
startMonth="01"
startYear=str(2022)
endMonth=str(12)
endYear=str(2022)
referencePeriods="20220101%2C20221001"

filter_names = ["Geography", "Type of Employee"]

url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ pickMembers1 + '&pickMembers%5B1%5D='+ pickMembers2 + '&cubeTimeFrame.startMonth='+ startMonth + '&cubeTimeFrame.startYear=' + startYear + '&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=' + referencePeriods


{'canada_salaried_employees_paid_a_fixed_salary': '{"January 2022":{"industrial_aggregate_including_unclassified_businesses":0.0,"industrial_aggregate_excluding_unclassified_businesses":6172688.0,"goods_producing_industries":771996.0,"forestry_logging_and_support":6101.0,"mining_quarrying_and_oil_and_gas_extraction":111448.0,"utilities":0.0,"construction":188782.0,"manufacturing":409149.0,"service_producing_industries":5400691.0,"trade":659266.0,"transportation_and_warehousing":265522.0,"information_and_cultural_industries":209652.0,"finance_and_insurance":596187.0,"real_estate_and_rental_and_leasing":113351.0,"professional_scientific_and_technical_services":657745.0,"management_of_companies_and_enterprises":0.0,"administrative_and_support_waste_management_and_remediation_services":212700.0,"educational_services":1236763.0,"health_care_and_social_assistance":261077.0,"arts_entertainment_and_recreation":69079.0,"accommodation_and_food_services":93938.0,"other_services_except_public_admi

In [6]:
print(test)

NameError: name 'test' is not defined

In [ ]:
print("Today: ", date.today())


Today:  2023-01-05


In [ ]:
test = scrapper.simple_scrapper()
print(test)

TypeError: simple_scrapper() missing 8 required positional arguments: 'pid', 'pickMembers1', 'pickMembers2', 'startMonth', 'startYear', 'endMonth', 'endYear', and 'referencePeriods'

In [ ]:
#url = "https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410020101&pickMembers%5B0%5D=1.3&pickMembers%5B1%5D=2.1&cubeTimeFrame.startMonth=06&cubeTimeFrame.startYear=2022&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=20220601%2C20221001"

#https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?


# pid=1410020101&
# pickMembers%5B0%5D=1.3
# &pickMembers%5B1%5D=2.1
# &cubeTimeFrame.startMonth=0
# 6&cubeTimeFrame.startYear=2022
# &cubeTimeFrame.endMonth=10
# &cubeTimeFrame.endYear=2022
# &referencePeriods=20220601%2C20221001b